# CrossFit Open Part 2: Scraping the Data

## Introduction

The CrossFit Games is a once year worldwide fitness competition that aims to find the fittest man and woman on Earth. Though the process to qualify for the CrossFit Games has changed over the years, the Crossfit Open is generally considered the first step to qualify. The CrossFit Open consists of five workouts over five weeks, with each workout released weekly on Thursday and scores due on Monday. After five weeks, the top competitors move on to secondary stages to qualify, or in some years can directly qualify for the CrossFit Games out of the Open. What makes Open unique is that anybody can sign up to participate. There are different versions of eah workout offered for different age groups and skill levels to allow the competition to allow more people the ability to participate. All workouts are either judged or videoed and scores are entered on a worldwide leaderboard for everyone to see their placing.

The Open began in 2011, and I personally have participated each year since 2016. Though I am not an athlete at a CrossFit Games level, I enjoy seeing how my fitness improves from year to year with my Open workout scores as concrete data points that show me what skills I need to improve on to be a more competitive and better athlete. As the sport of CrossFit grows and the overall abilities and fitness levels of athletes rise, I thought it would be an interesting project to pull old Open leaderboard data and create a tool that helps any Crossfit Open participant analyze what they need to work on to improve their scores and their fitness.

## Part 2: Loading the Data into PostgreSQL

In this notebook, I take all of the data that I scraped in <a href='https://github.com/megantoronto/crossfit-open/blob/main/crossfit_open_part_1.ipynb' >Part 1</a> and put it in a PostgreSQL database.

First, I imported all the necessary libraries.

In [6]:
import psycopg2
import os
import csv
import pandas as pd
import numpy as np

I set my local path:

In [2]:
path = os.path.abspath(os.getcwd())+'\\'

I first created a database on my local Postgres server called crossfit using pgAdmin, the GUI for Postgres. I then used the cell below to create a connection to the database:

In [41]:
conn = psycopg2.connect(
    host="localhost",
    database="crossfit",
    user="postgres",
    password=os.environ.get('DATABASE_PASSWORD'))

## 2020 and 2019 Data

I read in the data from one of the locally stored csv files for 2020 to in pull and print the column values:

In [8]:
df = pd.read_csv(path+'final_data\\open_2020_women.csv',low_memory=False)
for i in df.columns:
    print(i)

competitorId
competitorName
firstName
lastName
status
postCompStatus
gender
profilePicS3key
countryOfOriginCode
countryOfOriginName
divisionId
affiliateId
affiliateName
age
height
weight
highlight
countryChampion
ordinal_1
rank_1
score_1
scoreDisplay_1
mobileScoreDisplay_1
scoreIdentifier_1
scaled_1
video_1
breakdown_1
time_1
judge_1
affiliate_1
heat_1
lane_1
ordinal_2
rank_2
score_2
scoreDisplay_2
mobileScoreDisplay_2
scoreIdentifier_2
scaled_2
video_2
breakdown_2
time_2
judge_2
affiliate_2
heat_2
lane_2
ordinal_3
rank_3
score_3
scoreDisplay_3
mobileScoreDisplay_3
scoreIdentifier_3
scaled_3
video_3
breakdown_3
time_3
judge_3
affiliate_3
heat_3
lane_3
ordinal_4
rank_4
score_4
scoreDisplay_4
mobileScoreDisplay_4
scoreIdentifier_4
scaled_4
video_4
breakdown_4
time_4
judge_4
affiliate_4
heat_4
lane_4
ordinal_5
rank_5
score_5
scoreDisplay_5
mobileScoreDisplay_5
scoreIdentifier_5
scaled_5
video_5
breakdown_5
time_5
judge_5
affiliate_5
heat_5
lane_5
overallRank
overallScore


To create a SQL table, you have to assign a datatype to each column. To help save a bit of typing when assigning data these datatypes, I used the cell below:

In [15]:
types=['INT','INT','INT','TEXT','TEXT','TEXT','BOOLEAN','BOOLEAN','TEXT','REAL','TEXT','TEXT','INT','INT']*4
h=['ordinal_2', 'rank_2', 'score_2', 'scoreDisplay_2',
       'mobileScoreDisplay_2', 'scoreIdentifier_2', 'scaled_2', 'video_2',
       'breakdown_2', 'time_2', 'judge_2', 'affiliate_2', 'heat_2', 'lane_2',
       'ordinal_3', 'rank_3', 'score_3', 'scoreDisplay_3',
       'mobileScoreDisplay_3', 'scoreIdentifier_3', 'scaled_3', 'video_3',
       'breakdown_3', 'time_3', 'judge_3', 'affiliate_3', 'heat_3', 'lane_3',
       'ordinal_4', 'rank_4', 'score_4', 'scoreDisplay_4',
       'mobileScoreDisplay_4', 'scoreIdentifier_4', 'scaled_4', 'video_4',
       'breakdown_4', 'time_4', 'judge_4', 'affiliate_4', 'heat_4', 'lane_4',
       'ordinal_5', 'rank_5', 'score_5', 'scoreDisplay_5',
       'mobileScoreDisplay_5', 'scoreIdentifier_5', 'scaled_5', 'video_5',
       'breakdown_5', 'time_5', 'judge_5', 'affiliate_5', 'heat_5', 'lane_5']
for i in range(0,len(h)):
    print(h[i]+" "+types[i]+",")

ordinal_2 INT,
rank_2 INT,
score_2 INT,
scoreDisplay_2 TEXT,
mobileScoreDisplay_2 TEXT,
scoreIdentifier_2 TEXT,
scaled_2 BOOLEAN,
video_2 BOOLEAN,
breakdown_2 TEXT,
time_2 REAL,
judge_2 TEXT,
affiliate_2 TEXT,
heat_2 INT,
lane_2 INT,
ordinal_3 INT,
rank_3 INT,
score_3 INT,
scoreDisplay_3 TEXT,
mobileScoreDisplay_3 TEXT,
scoreIdentifier_3 TEXT,
scaled_3 BOOLEAN,
video_3 BOOLEAN,
breakdown_3 TEXT,
time_3 REAL,
judge_3 TEXT,
affiliate_3 TEXT,
heat_3 INT,
lane_3 INT,
ordinal_4 INT,
rank_4 INT,
score_4 INT,
scoreDisplay_4 TEXT,
mobileScoreDisplay_4 TEXT,
scoreIdentifier_4 TEXT,
scaled_4 BOOLEAN,
video_4 BOOLEAN,
breakdown_4 TEXT,
time_4 REAL,
judge_4 TEXT,
affiliate_4 TEXT,
heat_4 INT,
lane_4 INT,
ordinal_5 INT,
rank_5 INT,
score_5 INT,
scoreDisplay_5 TEXT,
mobileScoreDisplay_5 TEXT,
scoreIdentifier_5 TEXT,
scaled_5 BOOLEAN,
video_5 BOOLEAN,
breakdown_5 TEXT,
time_5 REAL,
judge_5 TEXT,
affiliate_5 TEXT,
heat_5 INT,
lane_5 INT,


Now I can copy and paste the column names I printed and what I printed above to make a function that creates a table for 2019 and 2020 data:

In [3]:
def create_table(table_name,conn):
    query = '''

    CREATE TABLE ''' + table_name+'''(
    competitorId INT,
    competitorName TEXT,
    firstName TEXT,
    lastName TEXT,
    status TEXT,
    postCompStatus TEXT,
    gender TEXT,
    profilePicS3key TEXT,
    countryOfOriginCode TEXT,
    countryOfOriginName TEXT,
    divisionId INT,
    affiliateId INT,
    affiliateName TEXT,
    age INT,
    height TEXT,
    weight TEXT,
    highlight BOOLEAN,
    countryChampion BOOLEAN,
    ordinal_1 INT,
    rank_1 INT,
    score_1 INT,
    scoreDisplay_1 TEXT,
    mobileScoreDisplay_1 TEXT,
    scoreIdentifier_1 TEXT,
    scaled_1 BOOLEAN,
    video_1 BOOLEAN,
    breakdown_1 TEXT,
    time_1 REAL,
    judge_1 TEXT,
    affiliate_1 TEXT,
    heat_1 TEXT,
    lane_1 TEXT,
    ordinal_2 INT,
    rank_2 INT,
    score_2 INT,
    scoreDisplay_2 TEXT,
    mobileScoreDisplay_2 TEXT,
    scoreIdentifier_2 TEXT,
    scaled_2 BOOLEAN,
    video_2 BOOLEAN,
    breakdown_2 TEXT,
    time_2 REAL,
    judge_2 TEXT,
    affiliate_2 TEXT,
    heat_2 TEXT,
    lane_2 TEXT,
    ordinal_3 INT,
    rank_3 INT,
    score_3 INT,
    scoreDisplay_3 TEXT,
    mobileScoreDisplay_3 TEXT,
    scoreIdentifier_3 TEXT,
    scaled_3 BOOLEAN,
    video_3 BOOLEAN,
    breakdown_3 TEXT,
    time_3 REAL,
    judge_3 TEXT,
    affiliate_3 TEXT,
    heat_3 TEXT,
    lane_3 TEXT,
    ordinal_4 INT,
    rank_4 INT,
    score_4 INT,
    scoreDisplay_4 TEXT,
    mobileScoreDisplay_4 TEXT,
    scoreIdentifier_4 TEXT,
    scaled_4 BOOLEAN,
    video_4 BOOLEAN,
    breakdown_4 TEXT,
    time_4 REAL,
    judge_4 TEXT,
    affiliate_4 TEXT,
    heat_4 TEXT,
    lane_4 TEXT,
    ordinal_5 INT,
    rank_5 INT,
    score_5 INT,
    scoreDisplay_5 TEXT,
    mobileScoreDisplay_5 TEXT,
    scoreIdentifier_5 TEXT,
    scaled_5 BOOLEAN,
    video_5 BOOLEAN,
    breakdown_5 TEXT,
    time_5 REAL,
    judge_5 TEXT,
    affiliate_5 TEXT,
    heat_5 TEXT,
    lane_5 TEXT,
    overallRank INT,
    overallScore INT
    );

    '''
    cur = conn.cursor()
    cur.execute(query)
    conn.commit()
    return "Table "+table_name+" has been created"

Now I need a function that will actually read in a csv and load the data into PostgreSQL to the table I create. I had to create a custom SQL COPY statement to handle all of the commas and new-line characters that are in the data:

In [23]:
def populate_table(table_name,csv_path,conn):
    f = open(csv_path, 'r',encoding='utf-8-sig')
    copy = '''
    COPY ''' + table_name + ''' FROM STDIN DELIMITER ',' CSV HEADER QUOTE '\"' ESCAPE '"';
    '''
    cur = conn.cursor()
    cur.copy_expert(copy,f)
    conn.commit()
    return "Table " + table_name + " has been populated"

Below I create and populate tables for the 2020 and 2019 men and women data:

In [12]:
create_table("open_women_2020",conn)

'Table open_women_2020 has been created'


In [2]:
populate_table("open_women_2020",path+'open_2020_women.csv',conn)

'Table open_women_2020 has been populated'


In [23]:
create_table("open_men_2020",conn)

'Table open_men_2020 has been created'

In [6]:
populate_table("open_men_2020",path+'open_2020_men.csv',conn)

'Table open_men_2020 has been populated'


In [50]:
create_table("open_women_2019",conn)

'Table open_women_2019 has been created'

In [8]:
populate_table("open_women_2019",path+'open_2019_women.csv',conn)

'Table open_women_2019 has been populated'


In [5]:
create_table("open_2019_men",conn)

'Table open_2019_men has been created'

In [9]:
populate_table("open_men_2019",path+'open_2019_men.csv',conn)

'Table open_men_2019 has been populated'


## 2018 Data

The 2018 CrossFit data has more columns than the 2019 and 2020 data, so the create_table function needs to be updated:

In [15]:
df_men_2018=pd.read_csv(path+'open_2018_men.csv',low_memory=False)
for i in df_men_2018.columns:
    print(i)

competitorId
competitorName
firstName
lastName
status
postCompStatus
gender
profilePicS3key
countryShortCode
regionalCode
regionId
regionName
divisionId
profession
affiliateId
affiliateName
age
height
weight
teamCaptain
highlight
ordinal_1
rank_1
score_1
scoreDisplay_1
mobileScoreDisplay_1
scoreIdentifier_1
scaled_1
video_1
time_1
breakdown_1
judge_1
affiliate_1
heat_1
lane_1
ordinal_2
rank_2
score_2
scoreDisplay_2
mobileScoreDisplay_2
scoreIdentifier_2
scaled_2
video_2
time_2
breakdown_2
judge_2
affiliate_2
heat_2
lane_2
ordinal_3
rank_3
score_3
scoreDisplay_3
mobileScoreDisplay_3
scoreIdentifier_3
scaled_3
video_3
time_3
breakdown_3
judge_3
affiliate_3
heat_3
lane_3
ordinal_4
rank_4
score_4
scoreDisplay_4
mobileScoreDisplay_4
scoreIdentifier_4
scaled_4
video_4
time_4
breakdown_4
judge_4
affiliate_4
heat_4
lane_4
ordinal_5
rank_5
score_5
scoreDisplay_5
mobileScoreDisplay_5
scoreIdentifier_5
scaled_5
video_5
time_5
breakdown_5
judge_5
affiliate_5
heat_5
lane_5
ordinal_6
rank_6
score_6


Additionally, some of the column datatypes needed to be reconverted to integers:

In [ ]:
cols=['divisionId','affiliateId','age','ordinal_1','rank_1','score_1','scaled_1','video_1',
         'ordinal_2','rank_2','score_2','scaled_2','video_2','ordinal_3','rank_3','score_3','scaled_3','video_3',
         'ordinal_4','rank_4','score_4','scaled_4','video_4','ordinal_5','rank_5','score_5','scaled_5','video_5',
         'ordinal_6','rank_6','score_6','scaled_6','video_6','overallRank','overallScore']

In [ ]:
def convert_to_int(df,cols):
    for i in cols:
        df[i]=df[i].astype('int')
    return df

In [ ]:
df_men_2018=convert_to_int(df_men_2018,cols)
df_men_2018.to_csv(path+'open_2018_men.csv',encoding='utf-8-sig',index=False)

In [ ]:
df_women_2018=pd.read_csv(path+'open_2018_women.csv')
df_women_2018=convert_to_int(df_women_2018)
df_women_2018.to_csv(path+'open_2018_women.csv',index=False)

Below I created a new function to create the tables for the 2018 data:

In [113]:
def create_table_2018(table_name,conn):
    query = '''

    CREATE TABLE ''' + table_name+'''(
    competitorId INT,
    competitorName TEXT,
    firstName TEXT,
    lastName TEXT,
    status TEXT,
    postCompStatus TEXT,
    gender TEXT,
    profilePicS3key TEXT,
    countryShortCode TEXT,
    regionalCode TEXT,
    regionId TEXT,
    regionName TEXT,
    divisionId INT,
    profession TEXT,
    affiliateId INT,
    affiliateName TEXT,
    age INT,
    height TEXT,
    weight TEXT,
    teamCaptain TEXT,
    highlight BOOLEAN,
    ordinal_1 INT,
    rank_1 INT,
    score_1 INT,
    scoreDisplay_1 TEXT,
    mobileScoreDisplay_1 TEXT,
    scoreIdentifier_1 TEXT,
    scaled_1 BOOLEAN,
    video_1 BOOLEAN,
    breakdown_1 TEXT,
    time_1 REAL,
    judge_1 TEXT,
    affiliate_1 TEXT,
    heat_1 TEXT,
    lane_1 TEXT,
    ordinal_2 INT,
    rank_2 INT,
    score_2 INT,
    scoreDisplay_2 TEXT,
    mobileScoreDisplay_2 TEXT,
    scoreIdentifier_2 TEXT,
    scaled_2 BOOLEAN,
    video_2 BOOLEAN,
    breakdown_2 TEXT,
    time_2 REAL,
    judge_2 TEXT,
    affiliate_2 TEXT,
    heat_2 TEXT,
    lane_2 TEXT,
    ordinal_3 INT,
    rank_3 INT,
    score_3 INT,
    scoreDisplay_3 TEXT,
    mobileScoreDisplay_3 TEXT,
    scoreIdentifier_3 TEXT,
    scaled_3 BOOLEAN,
    video_3 BOOLEAN,
    breakdown_3 TEXT,
    time_3 REAL,
    judge_3 TEXT,
    affiliate_3 TEXT,
    heat_3 TEXT,
    lane_3 TEXT,
    ordinal_4 INT,
    rank_4 INT,
    score_4 INT,
    scoreDisplay_4 TEXT,
    mobileScoreDisplay_4 TEXT,
    scoreIdentifier_4 TEXT,
    scaled_4 BOOLEAN,
    video_4 BOOLEAN,
    breakdown_4 TEXT,
    time_4 REAL,
    judge_4 TEXT,
    affiliate_4 TEXT,
    heat_4 TEXT,
    lane_4 TEXT,
    ordinal_5 INT,
    rank_5 INT,
    score_5 INT,
    scoreDisplay_5 TEXT,
    mobileScoreDisplay_5 TEXT,
    scoreIdentifier_5 TEXT,
    scaled_5 BOOLEAN,
    video_5 BOOLEAN,
    breakdown_5 TEXT,
    time_5 REAL,
    judge_5 TEXT,
    affiliate_5 TEXT,
    heat_5 TEXT,
    lane_5 TEXT,
    ordinal_6 INT,
    rank_6 INT,
    score_6 INT,
    scoreDisplay_6 TEXT,
    mobileScoreDisplay_6 TEXT,
    scoreIdentifier_6 TEXT,
    scaled_6 BOOLEAN,
    video_6 BOOLEAN,
    breakdown_6 TEXT,
    time_6 REAL,
    judge_6 TEXT,
    affiliate_6 TEXT,
    heat_6 TEXT,
    lane_6 TEXT,
    overallRank INT,
    overallScore INT
    );

    '''
    cur = conn.cursor()
    cur.execute(query)
    conn.commit()
    return "Table "+table_name+" has been created"

In [114]:
create_table_2018("open_2018_men",conn)

'Table open_2018_men has been created'

In [4]:
populate_table('open_2018_men',path+'open_2018_men.csv',conn)

'Table open_2018_men has been populated'


In [124]:
create_table_2018("open_2018_women",conn)

'Table open_2018_women has been created'

In [3]:
populate_table('open_2018_women',path+'open_2018_women.csv',conn)

'Table open_2018_women has been populated'


## 2017 Data

I read in the 2017 data to see the columns that are in the 2017 data:

In [10]:
df_men_2017=pd.read_csv(path+'open_2017_men.csv',low_memory=False)
for i in df_men_2017.columns:
    print(i)

competitorid
competitorname
regionid
affiliateid
divisionid
highlight
age
region
height
weight
profilepic
overallrank
overallscore
affiliate
division
workoutrank_1
workoutresult_1
scoreidentifier_1
scoredisplay_1
time_1
breakdown_1
judge_1
affiliate_1
video_1
workoutrank_2
workoutresult_2
scoreidentifier_2
scoredisplay_2
time_2
breakdown_2
judge_2
affiliate_2
video_2
workoutrank_3
workoutresult_3
scoreidentifier_3
scoredisplay_3
time_3
breakdown_3
judge_3
affiliate_3
video_3
workoutrank_4
workoutresult_4
scoreidentifier_4
scoredisplay_4
time_4
breakdown_4
judge_4
affiliate_4
video_4
workoutrank_5
workoutresult_5
scoreidentifier_5
scoredisplay_5
time_5
breakdown_5
judge_5
affiliate_5
video_5
nextstage
scaled_1
scaled_2
scaled_3
scaled_4
scaled_5


The 2017 data had different columns, so I created a new function to create a table for the 2017 data:

In [40]:
def create_table_2017(table_name,conn):
    query= '''
    CREATE TABLE ''' + table_name+'''(
    
    competitorid INT,
    competitorname TEXT,
    regionid INT,
    affiliateid INT,
    divisionid INT,
    highlight BOOLEAN,
    age INT,
    region TEXT,
    height TEXT,
    weight TEXT,
    profilepic TEXT,
    overallrank INT,
    overallscore INT,
    affiliate TEXT,
    division TEXT,
    workoutrank_1 INT,
    workoutresult_1 TEXT,
    scoreidentifier_1 TEXT,
    scoredisplay_1 TEXT,
    time_1 TEXT,
    breakdown_1 TEXT,
    judge_1 TEXT,
    affiliate_1 TEXT,
    video_1 TEXT,
    workoutrank_2 INT,
    workoutresult_2 TEXT,
    scoreidentifier_2 TEXT,
    scoredisplay_2 TEXT,
    time_2 TEXT,
    breakdown_2 TEXT,
    judge_2 TEXT,
    affiliate_2 TEXT,
    video_2 TEXT,
    workoutrank_3 INT,
    workoutresult_3 TEXT,
    scoreidentifier_3 TEXT,
    scoredisplay_3 TEXT,
    time_3 TEXT,
    breakdown_3 TEXT,
    judge_3 TEXT,
    affiliate_3 TEXT,
    video_3 TEXT,
    workoutrank_4 INT,
    workoutresult_4 TEXT,
    scoreidentifier_4 TEXT,
    scoredisplay_4 TEXT,
    time_4 TEXT,
    breakdown_4 TEXT,
    judge_4 TEXT,
    affiliate_4 TEXT,
    video_4 TEXT,
    workoutrank_5 INT,
    workoutresult_5 TEXT,
    scoreidentifier_5 TEXT,
    scoredisplay_5 TEXT,
    time_5 TEXT,
    breakdown_5 TEXT,
    judge_5 TEXT,
    affiliate_5 TEXT,
    video_5 TEXT,
    nextstage TEXT,
    scaled_1 BOOLEAN,
    scaled_2 BOOLEAN,
    scaled_3 BOOLEAN,
    scaled_4 BOOLEAN,
    scaled_5 BOOLEAN
    
    );
   
    '''
    
    cur = conn.cursor()
    cur.execute(query)
    conn.commit()
    return "Table "+table_name+" has been created"
    

In [22]:
create_table_2017("open_2017_men",conn)

'Table open_2017_men has been created'

In [35]:
populate_table('open_2017_men',path+'open_2017_men.csv',conn)

'Table open_2017_men has been populated'

In [38]:
create_table_2017("open_2017_women",conn)

'Table open_2017_women has been created'

In [43]:
populate_table('open_2017_women',path+'open_2017_women.csv',conn)

'Table open_2017_women has been populated'

Close the connection:

In [44]:
conn.close()

## 2016 Data

This notebook is a work in progress. I will be creating and populating a Postgres table for the 2016 data next!